using the ApproxBayes package to choose the optimal parameters for my amino acid model.

6 model parameters to be inferred

3 extra parameters to choose: epsilon, the perturbation kernel specs, and the number of particles to accept

In [9]:
using Pkg
Pkg.add("Stan")

 Resolving package versions...
 Installed StanDump ───────── v0.2.0
 Installed StanVariational ── v1.1.6
 Installed StanBase ───────── v1.1.6
 Installed StanSamples ────── v0.1.0
 Installed Observables ────── v0.2.3
 Installed BinDeps ────────── v1.0.0
 Installed StanDiagnose ───── v1.1.5
 Installed Stan ───────────── v5.1.0
 Installed FilePathsBase ──── v0.7.0
 Installed URIParser ──────── v0.4.0
 Installed Widgets ────────── v0.6.2
 Installed NearestNeighbors ─ v0.4.4
 Installed ElasticArrays ──── v1.0.0
 Installed StanOptimize ───── v1.1.5
 Installed Documenter ─────── v0.24.3
 Installed StatsPlots ─────── v0.12.0
 Installed StanSample ─────── v1.1.5
 Installed CSV ────────────── v0.5.22
 Installed Clustering ─────── v0.13.3
  Updating `~/.julia/environments/v1.3/Project.toml`
  [682df890] + Stan v5.1.0
  Updating `~/.julia/environments/v1.3/Manifest.toml`
  [9e28174c] + BinDeps v1.0.0
  [336ed68f] + CSV v0.5.22
  [aaaa29a8] + Clustering v0.13.3
  [e30172f5] + Documenter v0.24.3
  [

In [39]:
using ODE
using DifferentialEquations
using Pandas
using Plots
using Distributions
using ApproxBayes
using DiffEqBayes
using ModelingToolkit
using Stan
# using ParameterizedFunctions


ArgumentError: ArgumentError: Package ParameterizedFunctions not found in current path:
- Run `import Pkg; Pkg.add("ParameterizedFunctions")` to install the ParameterizedFunctions package.


In [40]:
# include("../code/multilevel-functions.jl");
# println("AA_simple and AA_popn_growth functions available")
include("setup_ABC.jl")
println("parameters setup complete")

parameters setup complete


In [55]:
time1= 10000.0
problm = ODEProblem(AA_simple,init,(0.,time1))
#the line above runs the single cell model with AA. no cell growth or nitrogenase
println("running with parameters $k_cat_AA-$k_a_NH4-$k_NH4-$k_a_AA-$k_NH4_AA-$k_ribo_a")
solved = ODE.solve(problm)
df1= DataFrame(solved)
println("burn in complete")

running with parameters 10.0-10.0-10.0-10.0-10.0-10.0
burn in complete


In [42]:
input2 = 10000.0
time2 = time1 + input2
endstate = size(solved,2)
init2 =solved[endstate]
problm2 = ODEProblem(AA_popn_growth,init2,(time1,time2));
# println("running AA popn growth")
# solved2 = solve(problm2)
# println("AA popn growth complete")

In [46]:
data = [1,2,3]
priors = [Uniform()]
abc_inference(problm2, Tsit5(), t, data, priors; ϵ=0.001,
    ABCalgorithm = ABCSMC)

UndefVarError: UndefVarError: priors not defined

In [14]:
init2

25-element Array{Float64,1}:
      1.0e11            
   4407.834024297853    
    321.535134369576    
   1615.4112048844129   
      7.370060728234548 
    321.53513436957667  
      7.370060728234535 
     20.0864436091661   
     20.086443609166057 
  70475.87231334136     
    143.07460198547028  
   4402.659254654273    
   3263.143246290681    
      1.6372852617557088
   1000.0               
      0.0               
      0.0               
      0.0               
      0.0               
      1.0               
  46152.566071084555    
 413176.0417655705      
  69912.92166676398     
   4367.491476055549    
   1602.507543639402    

In [9]:
a = init2[21]
AA = init2[22]
rmq = init2[4]
rmr = init2[2]
rmt = init2[5]
rmm = init2[7]
nit_mrna_ribo = init2[16]
AA_mrna_ribo = init2[25]

1602.507543639397

Order of molecules in solver:


s_out, rmr_0, em_0, rmq_0, rmt_0, et_0, rmm_0,
mt_0, mm_0, q_0, si_0, mq_0, mr_0, r_0, NH4_0, nit_mrna_0, nit_mrna_ribo_0, nit_0, exported_0, N_0, a_0, AA_0, AA_prot_0, AA_mrna_0, AA_mrna_ribo_0


In [10]:
gam= (gmax*a*AA)/(k_ribo_a*k_ribo_a_AA+k_ribo_a_AA*a+k_ribo_AA_a*AA+a*AA)
ttrate= (rmq + rmr + rmt + rmm + nit_mrna_ribo+AA_mrna_ribo)*gam
lam= ttrate/M #growth rate, lambda, is the total transcription rate over mid log cell mass


0.07758216572575495

In [11]:
sim_data = [AA, a, lam]

3-element Array{Float64,1}:
 413176.0417655678       
  46152.56607108405      
      0.07758216572575495

In [48]:
target_data = [9.6e8,1.8e10,0.1]#number of amino acids, ATP and growth rate

3-element Array{Float64,1}:
 9.6e8 
 1.8e10
 0.1   

In [13]:
function get_sim_data(params)
    k_cat_AA = params[1]
    k_a_NH4 = params[2]
    k_NH4 = params[3]
    k_a_AA = params[4]
    k_NH4_AA = params[5]
    k_ribo_a = params[6]
    
    
    problm = ODEProblem(AA_popn_growth,init2,(time1,20000.0));
    solved = solve(problm)

    
    numsteps = size(solved,2)
    state =solved[numsteps]
    a = state[21]
    AA = state[22]
    rmq = state[4]
    rmr = state[2]
    rmt = state[5]
    rmm = state[7]
    nit_mrna_ribo = state[16]
    AA_mrna_ribo = state[25]
    gam= (gmax*a*AA)/(k_ribo_a*k_ribo_a_AA+k_ribo_a_AA*a+k_ribo_AA_a*AA+a*AA)
    ttrate= (rmq + rmr + rmt + rmm + nit_mrna_ribo+AA_mrna_ribo)*gam
    lam= ttrate/M #growth rate, lambda, is the total transcription rate over mid log cell mass
    sim_data = [AA, a, lam]
    return sim_data
end

get_sim_data (generic function with 1 method)

In [14]:
function sum_sq_dist(sim_data, target_data)  
    output = (sim_data[1]-target_data[1])^2+(sim_data[2]-target_data[2])^2+(sim_data[3]-target_data[3])^2
    return output
end

sum_sq_dist (generic function with 1 method)

In [15]:
sum_sq_dist(sim_data, target_data)
# abs(sim_data[3]-target_data[3])

3.2491914538246577e20

In [16]:
# ABC1 = ABCSMC(AA_popn_growth, #simulation function
# 1, # num parameters
# 10000.0, #target for error
# Prior([Uniform(0.01, 10000)]); #prior for each of the parameters
# maxiterations = 10^6,
# )

In [17]:
#starting values for the 6 parameters to be inferred
k_cat_AA = 10.0
k_a_NH4 = 10.0
k_NH4 = 10.0
k_a_AA = 10.0
k_NH4_AA = 10.0
k_ribo_a = 10.0

10.0

In [18]:
f1 = @ode_def LotkaVolterraTest1 begin
 dx = a*x - x*y
 dy = -3*y + x*y
end a

p = [1.5]
u0 = [1.0,1.0]
tspan = (0.0,10.0)
prob1 = ODEProblem(f1,u0,tspan,p)

σ = 0.01                         # noise, fixed for now
t = collect(range(1,stop = 10,length =10))   # observation times
sol = solve(prob1,Tsit5())

randomized = VectorOfArray([(sol(t[i]) + σ * randn(2)) for i in 1:length(t)])
data = convert(Array,randomized)

bayesian_result_abc = abc_inference(prob1, Tsit5(), t, data, [Normal(1.5, 1)];
                                num_samples=500)

Preparing to run in serial on 1 processor
Preparing to run in serial on 1 processor
Total number of simulations: 1.95e+04
Cumulative number of simulations = [500, 6620, 10164, 12091, 13897, 15633, 17786, 19538]
Acceptance ratio: 2.56e-02
Tolerance schedule = [10.67, 5.93, 2.06, 0.65, 0.21, 0.07, 0.05, 0.05]

Median (95% intervals):
Parameter 1

: 1.50 (1.50,1.50)


In [51]:
using ApproxBayes

In [54]:
params = (k_cat_AA) #, k_a_NH4, k_NH4, k_a_AA, k_NH4_AA, k_ribo_a)
problm = ODEProblem(AA_popn_growth,init2,(time1,time2), params);
# solved = solve(problm)
priors = [Normal(5000,10000)]


ABC_result = abc_inference(problm, Tsit5(), time2, target_data, priors;
    num_samples = 500, ϵ = 1000, progress = true, ABCalgorithm = ABCSMC)

MethodError: MethodError: no method matching ABCSMC(::DiffEqBayes.var"#simfunc#22"{Base.Iterators.Pairs{Union{},Union{},Tuple{},NamedTuple{(),Tuple{}}},ODEProblem{Array{Float64,1},Tuple{Float64,Float64},true,Float64,ODEFunction{true,typeof(AA_popn_growth),LinearAlgebra.UniformScaling{Bool},Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,Nothing,Nothing},Base.Iterators.Pairs{Union{},Union{},Tuple{},NamedTuple{(),Tuple{}}},DiffEqBase.StandardODEProblem},Float64,typeof(Distances.euclidean),Tsit5}, ::Int64, ::Int64, ::Prior; nparticles=500, maxiterations=100000)
Closest candidates are:
  ABCSMC(::Function, ::Int64, !Matched::Float64, ::Prior; maxiterations, constants, nparticles, α, ϵ1, convergence, kernel) at /Users/leanne/.julia/packages/ApproxBayes/EK19t/src/types.jl:109

In [20]:
params

10.0

In [15]:
?DiffEqBayes.abc_inference

No documentation found.

`DiffEqBayes.abc_inference` is a `Function`.

```
# 1 method for generic function "abc_inference":
[1] abc_inference(prob::DiffEqBase.DEProblem, alg, t, data, priors; ϵ, distancefunction, ABCalgorithm, progress, num_samples, maxiterations, kwargs...) in DiffEqBayes at /Users/leanne/.julia/packages/DiffEqBayes/tTjJB/src/abc_inference.jl:17
```


In [16]:
?ABCSMC

search: ABCSMC ABCSMCModel GeneralizedBlackScholesProblem



```
ABCRejection(sim_func::Function, nparams::Int64, ϵT::Float64, prior::Prior; <keyword arguments>)
```

Create an ABCSMC type which will simulate data with sim*func. nparams is the number of parameters inputted into sim*func, ϵT is the target tolerance and prior sets the priors for the parameters. sim_func needs to take in 3 values, the parameters (in an array), constants (array) and target data in that order and needs to return 2 values, the first being the distance between the target data and simulated data and the second can be anything but is useful if for example you want to record some additional information about the simulations. ...

## Arguments

  * `maxiterations = 10^5`: Maximum number of samples before the ABC algorithm terminates.
  * `constants = []`: Any constants needed to simulate from sim_func
  * `nparticles = 100`: Number of particles (ie samples) of ABC algorithm
  * `α = 0.3`: The αth quantile of population i is chosen as the ϵ for population i + 1
  * `ϵ1 = 10^5`: Starting ϵ for first ABC SMC populations
  * `convergence = 0.05`: ABC SMC stops when ϵ in population i + 1 is within 0.05 of populations i
  * `kernel = uniformkernel`: Parameter perturbation kernel, default is a uniform distribution. `gaussiankernel` is also an option that is already available in ApproxBayes.jl. Alternatively you can code up your own kernel function. See kernels.jl for examples.

...


In [17]:
?range

search: range LinRange UnitRange StepRange StepRangeLen date_range bdate_range



```
range(start[, stop]; length, stop, step=1)
```

Given a starting value, construct a range either by length or from `start` to `stop`, optionally with a given step (defaults to 1, a [`UnitRange`](@ref)). One of `length` or `stop` is required.  If `length`, `stop`, and `step` are all specified, they must agree.

If `length` and `stop` are provided and `step` is not, the step size will be computed automatically such that there are `length` linearly spaced elements in the range (a [`LinRange`](@ref)).

If `step` and `stop` are provided and `length` is not, the overall range length will be computed automatically such that the elements are `step` spaced (a [`StepRange`](@ref)).

`stop` may be specified as either a positional or keyword argument.

!!! compat "Julia 1.1"
    `stop` as a positional argument requires at least Julia 1.1.


# Examples

```jldoctest
julia> range(1, length=100)
1:100

julia> range(1, stop=100)
1:100

julia> range(1, step=5, length=100)
1:5:496

julia> range(1, step=5, stop=100)
1:5:96

julia> range(1, 10, length=101)
1.0:0.09:10.0

julia> range(1, 100, step=5)
1:5:96
```

---

```
range(start::Color; stop::Color, length=100)
```

Generates `n`>2 colors in a linearly interpolated ramp from `start` to`stop`, inclusive, returning an `Array` of colors.

---

```
range(c::AbstractChains)
```

Returns the range used in a `Chains` object.


In [24]:
# using ParameterizedFunctions
f = @ode_def begin
  test_var = 22
  dy₁ = -k₁*y₁+k₃*y₂*y₃
  dy₂ =  k₁*y₁-k₂*y₂^2-k₃*y₂*y₃
  dy₃ =  k₂*y₂^2
end k₁ k₂ k₃

┌ Warning: Symbolic calculations could not initiate. Likely there's a function which is not differentiable by SymEngine.
└ @ ParameterizedFunctions C:\Users\leann\.julia\packages\ParameterizedFunctions\WcTrd\src\ode_def_opts.jl:244


#387 (generic function with 2 methods)

In [4]:
@variables x y z
@parameters σ ρ β
a = y - x
eqs = [0 ~ σ*a,
       0 ~ x*(ρ-z)-y,
       0 ~ x*y - β*z]
ns = NonlinearSystem(eqs, [x,y,z])
nlsys_func = generate_function(ns, [x,y,z], [σ,ρ,β])

(:((u, p)->begin
          #= C:\Users\leann\.julia\packages\ModelingToolkit\PPjhj\src\utils.jl:63 =#
          #= C:\Users\leann\.julia\packages\ModelingToolkit\PPjhj\src\utils.jl:64 =#
          if u isa Array
              #= C:\Users\leann\.julia\packages\ModelingToolkit\PPjhj\src\utils.jl:65 =#
              return #= C:\Users\leann\.julia\packages\ModelingToolkit\PPjhj\src\utils.jl:57 =# @inbounds(begin
                          #= C:\Users\leann\.julia\packages\ModelingToolkit\PPjhj\src\utils.jl:57 =#
                          let (x, y, z, σ, ρ, β) = (u[1], u[2], u[3], p[1], p[2], p[3])
                              [(*)(σ, (-)(y, x)), (-)((*)(x, (-)(ρ, z)), y), (-)((*)(x, y), (*)(β, z))]
                          end
                      end)
          else
              #= C:\Users\leann\.julia\packages\ModelingToolkit\PPjhj\src\utils.jl:67 =#
              X = #= C:\Users\leann\.julia\packages\ModelingToolkit\PPjhj\src\utils.jl:56 =# @inbounds(begin
                        

In [26]:
p = (1.0,1.0,1.0,10.0,0.001,100.0) # a,α,ubar,β,D1,D2
N = 100
Ax = Array(Tridiagonal([1.0 for i in 1:N-1],[-2.0 for i in 1:N],[1.0 for i in 1:N-1]))
Ay = copy(Ax)
Ax[2,1] = 2.0
Ax[end-1,end] = 2.0
Ay[1,2] = 2.0
Ay[end,end-1] = 2.0

a,α,ubar,β,D1,D2 = p
uss = (ubar+β)/α
vss = (a/β)*uss^2
r0 = zeros(100,100,2)
r0[:,:,1] .= uss.+0.1.*rand.()
r0[:,:,2] .= vss

function gm2!(dr,r,p,t)
  a,α,ubar,β,D1,D2 = p
  u = @view r[:,:,1]
  v = @view r[:,:,2]
  du = @view dr[:,:,1]
  dv = @view dr[:,:,2]
  Du = D1*(Ay*u + u*Ax)
  Dv = D2*(Ay*v + v*Ax)
  @. du = Du + a.*u.*u./v + ubar - α*u
  @. dv = Dv + a.*u.*u - β*v
end
prob = ODEProblem(gm2!,r0,(0.0,0.1),p)

UndefVarError: UndefVarError: Tridiagonal not defined

In [18]:
include("setup_ABC.jl")
# println("parameters setup complete")

# s_out(t) rmr(t) em(t) rmq(t) rmt(t) et(t) rmm(t) mt(t) mm(t) q(t) si(t) mq(t) mr(t) r(t) NH4(t) nit_mrna(t) nit_mrna_ribo(t) nit(t) NH4_ex(t) N(t) a(t) AA(t) AA_prot(t) AA_mrna(t) AA_mrna_ribo(t)
@variables s_out rmr em rmq rmt et rmm mt mm q si mq mr r NH4 nit_mrna nit_mrna_ribo nit NH4_ex N a AA AA_prot AA_mrna AA_mrna_ribo

@parameters k_ribo_a_AA k_ribo_AA_a k_a k_cat_AA k_a_NH4 k_NH4 k_a_AA k_NH4_AA k_ribo_a

gam= (gmax*a*AA)/(k_ribo_a*k_ribo_a_AA+k_ribo_a_AA*a+k_ribo_AA_a*AA+a*AA) #updated gamma equation that uses both ATP and AA
ttrate= (rmq + rmr + rmt + rmm + nit_mrna_ribo+AA_mrna_ribo)*gam #total translation rate (sum of the mRNA/ribosome complexes times translation rate)
lam= ttrate/M
nucat= em*vm*si/(Km + si) 
AA_vo = ((k_cat_AA*2*a*NH4)/(k_a_NH4*k_a))/(1+(1+(AA/k_a_AA))+(NH4/k_a_NH4)+(1+(AA/k_NH4_AA))+(NH4/k_NH4))
new_AA = AA_vo*AA_prot
AA_a_use = new_AA*2 #each AA produced uses x number of ATP molecules
AA_NH4_use = new_AA*2 #each AA produced uses 2 NH4 molecules
export_rate = 0 #export rate of NH4
fixation = 0
death_rate = 0
   
eqs = [
    s_out~ ((-et*vt*s_out/(Kt+s_out))*N)-ds*s_out+(export_rate*NH4*50)+1e7
    rmr~ kb*r*mr-ku*rmr-(gam/nr)*rmr-lam*rmr
    em~ (gam/nx)*rmm-lam*em
    rmq~ kb*r*mq-ku*rmq-(gam/nx)*rmq-lam*rmq
    rmt~ kb*r*mt-ku*rmt-gam/nx*rmt-lam*rmt
    et~ (gam/nx)*rmt-lam*et
    rmm~ kb*r*mm-ku*rmm-gam/nx*rmm-lam*rmm
    mt~ (we*a/(thetax+a))+ku*rmt+(gam/nx)*rmt-kb*r*mt-dm*mt-lam*mt
    mm~ (we*a/(thetax+a))+ku*rmm+(gam/nx)*rmm-kb*r*mm-dm*mm-lam*mm
    q~ (gam/nx)*rmq-lam*q
    si~ (et*vt*s_out/(Kt+s_out))-nucat-lam*si
    mq~ (wq*a/(thetax+a)/(1+(q/Kq)^nq))+ku*rmq+(gam/nx)*rmq-kb*r*mq-dm*mq-lam*mq
    mr~ (wr*a/(thetar+a))+ku*rmr+(gam/nr)*rmr-kb*r*mr-dm*mr-lam*mr
    r~ ku*rmr+ku*rmt+ku*rmm+ku*rmq+(gam/nr)*rmr+(gam/nr)*rmr+(gam/nx)*rmt+(gam/nx)*rmm+(gam/nx)*rmq-kb*r*mr-kb*r*mt-kb*r*mm-kb*r*mq+ku*nit_mrna_ribo+(gam/nx)*nit_mrna_ribo-kb*r*nit_mrna+ku*AA_mrna_ribo+(gam/nx)*AA_mrna_ribo-kb*r*AA_mrna-lam*r
    NH4~ fixation*nit-export_rate*NH4-lam*NH4-AA_NH4_use+2.3e6
    nit_mrna~ 0 #(v_nit*a/(thetax + a))+(ku*nit_mrna_ribo)+(gam/nx*nit_mrna_ribo)-(kb*r*nit_mrna)-(dm*nit_mrna)-(lam*nit_mrna) #num of free nitrogenase coding mRNA
    nit_mrna_ribo~ 0 #(kb*r*nit_mrna)-(ku*nit_mrna_ribo)-(gam/nx*nit_mrna_ribo)-(lam*nit_mrna_ribo) #num of nit mRNA-ribosome complexes
    nit~ 0 #(gam/nx)*nit_mrna_ribo-lam*nit #num of nitrogenase proteins
    NH4_ex~ 0 #(export_rate*NH4)#total num of NH4 molecules exported this timestep
    N~ lam*N-death_rate*N #the change in num of bacterial cells instead of just loss to dilution
    a~ ns*nucat-ttrate-fixation*nit-AA_a_use-lam*a
    AA~ new_AA-ttrate-lam*AA
    AA_prot~ (gam/nx)*AA_mrna_ribo-lam*AA_prot
    AA_mrna~ ((w_AA*a/(thetax + a)))+(ku*AA_mrna_ribo)+(gam/nx*AA_mrna_ribo)-(kb*r*AA_mrna)-(dm*AA_mrna)-(lam*AA_mrna)
    AA_mrna_ribo~ (kb*r*AA_mrna)-(ku*AA_mrna_ribo)-(gam/nx*AA_mrna_ribo)-(lam*AA_mrna_ribo)
]

#     end k_cat_AA k_a_NH4 k_NH4 k_a_AA k_NH4_AA k_ribo_a



25-element Array{Equation,1}:
 Equation(s_out, (((((-et * 726.0) * s_out) / (1000.0 + s_out)) * N - 0.01s_out) + (0NH4) * 50) + 1.0e7)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [28]:
# problm3 = ODEProblem(eqs,init2,(time1,time2),params)

de = ODESystem(eqs, [s_out, rmr, em, rmq, rmt, et, rmm, mt, mm, q, si, mq, mr, r, NH4, nit_mrna, nit_mrna_ribo, nit, NH4_ex, N, a, AA, AA_prot, AA_mrna, AA_mrna_ribo],[k_ribo_a_AA, k_ribo_AA_a, k_a, k_cat_AA, k_a_NH4, k_NH4, k_a_AA, k_NH4_AA, k_ribo_a])

MethodError: MethodError: no method matching ODESystem(::Array{Equation,1}, ::Array{Operation,1}, ::Array{Operation,1})
Closest candidates are:
  ODESystem(::AbstractArray{#s43,1} where #s43<:Equation, ::Any, ::Any, !Matched::Any) at /Users/leanne/.julia/packages/ModelingToolkit/PPjhj/src/systems/diffeqs/diffeqsystem.jl:109
  ODESystem(::Any, ::Any, ::Any, !Matched::Any, !Matched::Any, !Matched::Any, !Matched::Any) at /Users/leanne/.julia/packages/ModelingToolkit/PPjhj/src/systems/diffeqs/diffeqsystem.jl:59
  ODESystem(!Matched::AbstractArray{ModelingToolkit.DiffEq,1}, ::Any, ::Any, !Matched::Any) at /Users/leanne/.julia/packages/ModelingToolkit/PPjhj/src/systems/diffeqs/diffeqsystem.jl:102
  ...

In [29]:
type(eqs)

UndefVarError: UndefVarError: type not defined

In [26]:
?AbstractArray

search: AbstractArray AbstractDiffEqArray AbstractVectorOfArray



```
AbstractArray{T,N}
```

Supertype for `N`-dimensional arrays (or array-like types) with elements of type `T`. [`Array`](@ref) and other types are subtypes of this. See the manual section on the [`AbstractArray` interface](@ref man-interface-array).


In [23]:
@parameters t σ ρ β
@variables x(t) y(t) z(t)
@derivatives D'~t


eqs = [D(x) ~ σ*(y-x),
       D(y) ~ x*(ρ-z)-y,
       D(z) ~ x*y - β*z]

3-element Array{Equation,1}:
 Equation(derivative(x(t), t), σ * (y(t) - x(t)))       
 Equation(derivative(y(t), t), x(t) * (ρ - z(t)) - y(t))
 Equation(derivative(z(t), t), x(t) * y(t) - β * z(t))  

In [24]:
de = ODESystem(eqs, t, [x,y,z], [σ,ρ,β])


ODESystem(ModelingToolkit.DiffEq[ModelingToolkit.DiffEq(x, 1, σ * (y(t) - x(t))), ModelingToolkit.DiffEq(y, 1, x(t) * (ρ - z(t)) - y(t)), ModelingToolkit.DiffEq(z, 1, x(t) * y(t) - β * z(t))], t, Variable[x, y, z], Variable[σ, ρ, β], Base.RefValue{Array{Expression,2}}(Array{Expression}(undef,0,0)), Base.RefValue{Array{Expression,2}}(Array{Expression}(undef,0,0)), Base.RefValue{Array{Expression,2}}(Array{Expression}(undef,0,0)))

In [20]:
t = collect(linspace(1,10000))
data = [1,2,3]
abc_inference(eqs, Tsit5(),t,data,[Uniform()]

UndefVarError: UndefVarError: linspace not defined

In [18]:
?DEProblem

search: ODEProblem RODEProblem SplitODEProblem DynamicalODEProblem



```julia
struct ODEProblem{uType, tType, isinplace, P, F, K, PT} <: DiffEqBase.AbstractODEProblem{uType,tType,isinplace}
```

Defines an ODE problem.

# Fields

  * `f`

    The ODE is `du/dt = f(u,p,t)`.
  * `u0`

    The initial condition is `u(tspan[1]) = u0`.
  * `tspan`

    The solution `u(t)` will be computed for `tspan[1] ≤ t ≤ tspan[2]`.
  * `p`

    Constant parameters to be supplied as the second argument of `f`.
  * `kwargs`

    A callback to be applied to every solver which uses the problem.
  * `problem_type`

---

```
ODEProblem(f::ODEFunction,u0,tspan,p=NullParameters(),callback=CallbackSet())
```

Define an ODE problem from an [`ODEFunction`](@ref).


In [19]:
?ODEFunction

search: ODEFunction RODEFunction DynamicalODEFunction



```julia
struct ODEFunction{iip, F, TMM, Ta, Tt, TJ, JVP, VJP, JP, TW, TWt, TPJ, S, TCV} <: DiffEqBase.AbstractODEFunction{iip}
```

---

Create an `ODEFunction` from the [`ODESystem`](@ref). The arguments `dvs` and `ps` are used to set the order of the dependent variable and parameter vectors, respectively.


In [20]:
?ODESystem

search: ODESystem



```julia
struct ODESystem <: ModelingToolkit.AbstractSystem
```

A system of ordinary differential equations.

# Fields

  * `eqs` - The ODEs defining the system.

# Examples

```
using ModelingToolkit

@parameters t σ ρ β
@variables x(t) y(t) z(t)
@derivatives D'~t

eqs = [D(x) ~ σ*(y-x),
       D(y) ~ x*(ρ-z)-y,
       D(z) ~ x*y - β*z]

de = ODESystem(eqs)
```


In [16]:
ABC_func = @ode_def begin
    gam= (gmax*a*AA)/(k_ribo_a*k_ribo_a_AA+k_ribo_a_AA*a+k_ribo_AA_a*AA+a*AA) #updated gamma equation that uses both ATP and AA
    ttrate= (rmq + rmr + rmt + rmm + nit_mrna_ribo+AA_mrna_ribo)*gam #total translation rate (sum of the mRNA/ribosome complexes times translation rate)
    lam= ttrate/M
    nucat= em*vm*si/(Km + si) #nucat is the num of internal substrate molecules used by catabolism. em is the num of molecules of metabolic enzyme, vm is the rate of the metabolic enzyme and si is the internal substrate level
    AA_vo = ((k_cat_AA*2*a*NH4)/(k_a_NH4*k_a))/(1+(1+(AA/k_a_AA))+(NH4/k_a_NH4)+(1+(AA/k_NH4_AA))+(NH4/k_NH4))
    new_AA = AA_vo*AA_prot
    AA_a_use = new_AA*2 #each AA produced uses x number of ATP molecules
    AA_NH4_use = new_AA*2 #each AA produced uses 2 NH4 molecules
    export_rate = 0 #export rate of NH4
    
    ds_out= ((-et*vt*s_out/(Kt + s_out))*N)-ds*s_out +(export_rate*NH4*50) +1e7 #change in amount of substrate, a proxy for amount of host plant
    drmr= kb*r*mr-ku*rmr-(gam/nr)*rmr-lam*rmr#-dp*r
    dem= (gam/nx)*rmm-lam*em#-dp*em
    drmq= kb*r*mq-ku*rmq-(gam/nx)*rmq-lam*rmq
    drmt= kb*r*mt-ku*rmt-gam/nx*rmt-lam*rmt
    det= (gam/nx)*rmt-lam*et#-dp*et
    drmm= kb*r*mm-ku*rmm-gam/nx*rmm-lam*rmm
    dmt= (we*a/(thetax + a))+ku*rmt+(gam/nx)*rmt-kb*r*mt-dm*mt-lam*mt
    dmm= (we*a/(thetax + a))+ku*rmm+(gam/nx)*rmm-kb*r*mm-dm*mm-lam*mm
    dq= (gam/nx)*rmq-lam*q#-dp*q
    dsi= (et*vt*s_out/(Kt + s_out))-nucat-lam*si#-degredation*si
    dmq= (wq*a/(thetax + a)/(1 + (q/Kq)^nq))+ku*rmq+(gam/nx)*rmq-kb*r*mq-dm*mq-lam*mq
    dmr= (wr*a/(thetar + a))+ku*rmr+(gam/nr)*rmr-kb*r*mr-dm*mr-lam*mr
    dr= ku*rmr +ku*rmt+ ku*rmm+ ku*rmq +(gam/nr)*rmr+(gam/nr)*rmr +(gam/nx)*rmt +(gam/nx)*rmm +(gam/nx)*rmq -kb*r*mr- kb*r*mt- kb*r*mm -kb*r*mq +ku*nit_mrna_ribo +(gam/nx)*nit_mrna_ribo-kb*r*nit_mrna +ku*AA_mrna_ribo +(gam/nx)*AA_mrna_ribo-kb*r*AA_mrna -lam*r
    
    dNH4= fixation*nit-export_rate*NH4 -lam*NH4 -AA_NH4_use + 2.3e6
    dnit_mrna= 0 #(v_nit*a/(thetax + a))+(ku*nit_mrna_ribo)+(gam/nx*nit_mrna_ribo)-(kb*r*nit_mrna)-(dm*nit_mrna)-(lam*nit_mrna) #num of free nitrogenase coding mRNA
    dnit_mrna_ribo= 0 #(kb*r*nit_mrna)-(ku*nit_mrna_ribo)-(gam/nx*nit_mrna_ribo)-(lam*nit_mrna_ribo) #num of nit mRNA-ribosome complexes
    dnit= 0 #(gam/nx)*nit_mrna_ribo-lam*nit #num of nitrogenase proteins
    dNH4_ex= 0 #(export_rate*NH4)#total num of NH4 molecules exported this timestep
    dN=lam*N-death_rate*N #the change in num of bacterial cells, instead of just loss to dilution
    da= ns*nucat-ttrate-fixation*nit-AA_a_use-lam*a
    
    dAA= new_AA -ttrate-lam*AA #num of AA
    dAA_prot=(gam/nx)*AA_mrna_ribo-lam*AA_prot #num of AA making proteins
    dAA_mrna=((w_AA*a/(thetax + a)))+(ku*AA_mrna_ribo)+(gam/nx*AA_mrna_ribo)-(kb*r*AA_mrna)-(dm*AA_mrna)-(lam*AA_mrna)#num of free AA mRNAs
    dAA_mrna_ribo=(kb*r*AA_mrna)-(ku*AA_mrna_ribo)-(gam/nx*AA_mrna_ribo)-(lam*AA_mrna_ribo)#num of AA mRNAs bound to ribosomes
    
    end k_cat_AA k_a_NH4 k_NH4 k_a_AA k_NH4_AA k_ribo_a

LoadError: UndefVarError: @ode_def not defined